<a href="https://colab.research.google.com/github/ehsanre1376/AI-ML-Cryptocurrency-Trade-Bot/blob/main/Test_API_Read_Data_BTC_1m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ccxt pandas matplotlib ta requests

In [ ]:
# First install required libraries:
# pip install requests pandas matplotlib ta

import requests
import pandas as pd
import matplotlib.pyplot as plt
from ta import add_all_ta_features

def get_crypto_data(coin_id='bitcoin', vs_currency='usd', days='max'):
    """Fetch historical crypto data from CoinGecko API"""
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
    params = {
        'vs_currency': vs_currency,
        'days': days,
        'interval': 'daily'
    }

    response = requests.get(url, params=params)
    data = response.json()

    # Create DataFrame
    df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    return df

# Fetch Bitcoin data
btc = get_crypto_data('bitcoin', days=365)

# Calculate technical indicators using ta library
btc = add_all_ta_features(
    btc, open="price", high="price", low="price", close="price", volume=None
)

# Simple moving averages
btc['SMA_20'] = btc['price'].rolling(window=20).mean()
btc['SMA_50'] = btc['price'].rolling(window=50).mean()

# Plot price data
plt.figure(figsize=(12, 6))
plt.plot(btc['price'], label='Price')
plt.plot(btc['SMA_20'], label='20-day SMA')
plt.plot(btc['SMA_50'], label='50-day SMA')
plt.title('Bitcoin Price with Moving Averages')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()

# Display technical indicators
print(btc[['price', 'trend_macd', 'momentum_rsi', 'volatility_bbh', 'volatility_bbl']].tail())

# Save data to CSV
btc.to_csv('bitcoin_analysis.csv')

In [ ]:
# Install dependencies: pip install requests pandas matplotlib ta

import requests
import pandas as pd
import matplotlib.pyplot as plt
from ta import add_all_ta_features

def get_crypto_data(coin_id='bitcoin', vs_currency='usd', days=365):
    """Fetch OHLC data with volume from CoinGecko API"""
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
    params = {
        'vs_currency': vs_currency,
        'days': days,
        'interval': 'daily'
    }

    response = requests.get(url, params=params)
    data = response.json()

    # Create DataFrame with OHLC (using closing price for simplicity)
    df = pd.DataFrame(data['prices'], columns=['timestamp', 'close'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Add volume data
    df['volume'] = pd.DataFrame(data['total_volumes'], columns=['timestamp', 'volume'])['volume']

    # For this API, we'll assume OHLC = closing price (common daily approximation)
    df['open'] = df['close']
    df['high'] = df['close']
    df['low'] = df['close']

    df.set_index('timestamp', inplace=True)
    return df

# Fetch data
btc = get_crypto_data('bitcoin', days=365)

# Calculate technical indicators (now with proper column names)
btc = add_all_ta_features(
    btc,
    open="open",
    high="high",
    low="low",
    close="close",
    volume="volume"
)

# Simple moving averages
btc['SMA_20'] = btc['close'].rolling(window=20).mean()
btc['SMA_50'] = btc['close'].rolling(window=50).mean()

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(btc['close'], label='Price')
plt.plot(btc['SMA_20'], label='20-day SMA')
plt.plot(btc['SMA_50'], label='50-day SMA')
plt.title('Bitcoin Price Analysis')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()

# Display indicators
print(btc[['close', 'trend_macd', 'momentum_rsi', 'volatility_bbh', 'volatility_bbl']].tail())

# Save data
btc.to_csv('crypto_analysis.csv')

In [ ]:
# Install: pip install yfinance pandas

import yfinance as yf
import pandas as pd

def get_yahoo_crypto_data(symbol='BTC-USD', period='1d', interval='1m'):
    """Get crypto data from Yahoo Finance"""
    crypto = yf.Ticker(symbol)
    data = crypto.history(period=period, interval=interval)
    return data

# Get Bitcoin data (last 1 day with 1-minute intervals)
btc = get_yahoo_crypto_data('BTC-USD', period='1d', interval='2m')

# Clean data
btc = btc[['Open', 'High', 'Low', 'Close', 'Volume']]
btc.columns = [col.lower() for col in btc.columns]

print(btc.head())


In [ ]:
# Install first: pip install yfinance pandas matplotlib

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# 1. Get Bitcoin data with 1-minute intervals (last 7 days)
btc = yf.Ticker("BTC-USD").history(period="7d", interval="1m")

# 2. Clean and format data
btc = btc[['Open', 'High', 'Low', 'Close', 'Volume']]
btc.columns = [col.lower() for col in btc.columns]
btc.index.name = 'timestamp'

# 3. Display readable format in console
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
print("Latest Bitcoin Price Data:")
print(btc)
print("\nSummary Statistics:")
print(btc.describe())

# 4. Save to CSV file
csv_filename = 'bitcoin_1min_data.csv'
btc.to_csv(csv_filename)
print(f"\nData saved to {csv_filename} in your current working directory")

# 5. Visualize closing prices
plt.figure(figsize=(12, 6))
plt.plot(btc['close'], label='BTC/USD Price')
plt.title('Bitcoin Price (1-Minute Intervals)')
plt.xlabel('Timestamp')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()